In [2]:
import math

In [3]:
#Value Object.
class Value:

  def __init__(self, data, _children=(), _op='', label=''):
    self.data = data
    self.grad = 0.0
    self._backward = lambda: None
    self._prev = set(_children)
    self._op = _op
    self.label = label

  def __repr__(self):
    return f"Value(data={self.data})"

  def __add__(self, other):
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data + other.data, (self, other), '+')

    def _backward():
      self.grad += 1.0 * out.grad
      other.grad += 1.0 * out.grad
    out._backward = _backward

    return out

  def __mul__(self, other):
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data * other.data, (self, other), '*')

    def _backward():
      self.grad += other.data * out.grad
      other.grad += self.data * out.grad
    out._backward = _backward

    return out

  def __pow__(self, other):
    assert isinstance(other, (int, float)), "only supporting int/float powers for now"
    out = Value(self.data**other, (self,), f'**{other}')

    def _backward():
        self.grad += other * (self.data ** (other - 1)) * out.grad
    out._backward = _backward

    return out

  def __rmul__(self, other): # other * self
    return self * other

  def __truediv__(self, other): # self / other
    return self * other**-1

  def __neg__(self): # -self
    return self * -1

  def __sub__(self, other): # self - other
    return self + (-other)

  def __radd__(self, other): # other + self
    return self + other

  def tanh(self):
    #print(type(self))
    x = self.data
    #print(type(x))
    t = (math.exp(2*x) - 1)/(math.exp(2*x) + 1)
    out = Value(t, (self, ), 'tanh')

    def _backward():
      self.grad += (1 - t**2) * out.grad
    out._backward = _backward

    return out

  def exp(self):
    x = self.data
    out = Value(math.exp(x), (self, ), 'exp')

    def _backward():
      self.grad += out.data * out.grad # NOTE: in the video I incorrectly used = instead of +=. Fixed here.
    out._backward = _backward

    return out


  def backward(self):

    topo = []
    visited = set()
    def build_topo(v):
      if v not in visited:
        visited.add(v)
        for child in v._prev:
          build_topo(child)
        topo.append(v)
    build_topo(self)

    self.grad = 1.0
    for node in reversed(topo):
      node._backward()

In [ ]:
#Test Space to see if the calculations work
a = Value(2.0)
print(2 * a)

In [11]:
#Code from Andrej's video to help visualize the connections and the graph network
from graphviz import Digraph

def trace(root):
  # builds a set of all nodes and edges in a graph
  nodes, edges = set(), set()
  def build(v):
    if v not in nodes:
      nodes.add(v)
      for child in v._prev:
        edges.add((child, v))
        build(child)
  build(root)
  return nodes, edges

def draw_dot(root):
  dot = Digraph(format='svg', graph_attr={'rankdir': 'LR'}) # LR = left to right

  nodes, edges = trace(root)
  for n in nodes:
    uid = str(id(n))
    # for any value in the graph, create a rectangular ('record') node for it
    dot.node(name = uid, label = "{ %s | data %.4f | grad %.4f }" % (n.label, n.data, n.grad), shape='record')
    if n._op:
      # if this value is a result of some operation, create an op node for it
      dot.node(name = uid + n._op, label = n._op)
      # and connect this node to it
      dot.edge(uid + n._op, uid)

  for n1, n2 in edges:
    # connect n1 to the op node of n2
    dot.edge(str(id(n1)), str(id(n2)) + n2._op)

  return dot

In [158]:
# Test to see the graph and it's connections
# inputs x1,x2
x1 = Value(2.0, label='x1')
x2 = Value(0.0, label='x2')
# weights w1,w2
w1 = Value(-3.0, label='w1')
w2 = Value(1.0, label='w2')
# bias of the neuron
b = Value(6.8813735870195432, label='b')
# x1*w1 + x2*w2 + b
x1w1 = x1*w1; x1w1.label = 'x1*w1'
x2w2 = x2*w2; x2w2.label = 'x2*w2'
x1w1x2w2 = x1w1 + x2w2; x1w1x2w2.label = 'x1*w1 + x2*w2'
n = x1w1x2w2 + b; n.label = 'n'
o = n.tanh()
o.label = 'o'
o.backward()

In [ ]:
draw_dot(o)

In [4]:
#Neuron Class with all the necessary functions for creating an MLP
import random
class Neuron:
  def __init__(self, nin):
    self.weights = [Value(random.uniform(-1,1)) for i in range(nin)]
    self.bias = Value(random.uniform(-1, 1))

  def __call__(self, x):
    res = zip(self.weights, x)
    total = Value(0)
    for i, j in res:
      total += j * i
    total += self.bias
    out = total.tanh()
    return out

  def parameters(self):
    return self.weights + [self.bias]
class Layer:
  def __init__(self, nin, nouts):
    self.neurons = [Neuron(nin) for i in range(nouts)]

  def __call__(self, x):
    outs = [n(x) for n in self.neurons]
    return outs[0] if len(outs) == 1 else outs

  def parameters(self):
    params = []
    for n in self.neurons:
      params += n.parameters()
    return params


class MLP:
  def __init__(self, nin, nouts):
    sz = [nin] + nouts
    self.layers = [Layer(sz[i], sz[i+1]) for i in range(len(nouts))]

  def __call__(self, x):
    for layer in self.layers:
      x = layer(x)
    return x

  def parameters(self):
    params = []
    for layer in self.layers:
      params += layer.parameters()
    return params

# x = [2.0, 3.0]
# node = Neuron(2)
# print(node(x))

In [21]:
x = [2.0, 3.0, -1.0]
n = MLP(784, [3, 3, 1])

In [969]:
#Small test data set that Andrej created in his video
xs = [[2.0, 3.0, -1.0],
      [3.0, -1.0, 5.0],
      [0.5, 1.0, -2.0],
      [1.0, 1.0, -1.0]]

ys = [1.0, -1.0, -1.0, 1.0]

In [ ]:
#Entire forward and backward pass process
for k in range(50):
  ypred = [n(x) for x in xs]
  loss = sum((act-pred)**2 for act, pred in zip(ypred, ys))
  loss.backward()
  for p in n.parameters():
    p.data += -0.01 * p.grad
  print(k, loss)
  print(ypred)
  break


# MNIST CODE
This process still isn't entirely a 100 percent. The values encounter overflow exceptions sometimes which I still need to fix, but here is my progress thus far.

In [ ]:
# Read in MNIST data
import csv
labels = []
train_set = []
with open('mnist_train.csv', newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        vals = list(row.values())
        label = vals.pop(0)
        labels.append(int(label))

        train_set.append([int(i) for i in vals])
print(len(train_set[0]))

In [ ]:
#Forward and backward pass process for the MNIST data
for k in range(50):
  ypred = [n(x) for x in training]
  loss = sum((act-pred)**2 for act, pred in zip(ypred, labels))
  loss.backward()
  for p in n.parameters():
   p.data += -0.05 * p.grad
  print(k, loss)
  print(ypred)


In [ ]:
#Perform scaling on the data
import numpy as np
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()
min_max_scaler2 = preprocessing.MinMaxScaler()
train_set = min_max_scaler.fit_transform(train_set)
labels = min_max_scaler2.fit_transform(labels)

In [23]:
#Attempted to reduce data size because too much RAM was being utilized
training = train_set[0:100]
print(len(training))
labels = labels[0:100]

100
